## SET UP

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/LauraMJanssen/espcn.git
%cd espcn

In [ ]:
!pip install keras-tqdm
!sudo pip install scikit-video

## TRAIN

In [ ]:
!python3 train.py --train /content/gdrive/MyDrive/MA/IVUSImages_PNG_Train2 \
                  --validation /content/gdrive/MyDrive/MA/IVUSImages_PNG_Valid2 \
                  --test /content/gdrive/MyDrive/MA/IVUSImages_PNG_Valid2 \
                  --scale 4 \
                  --stage default \
                  --steps_per_epoch 677 \
                  --epochs 40 

In [ ]:
!zip -r /content/ESRCN_model.zip /content/ESPCN-Keras/model/ESPCN_4X.h5

  adding: content/ESPCN-Keras/model/ESPCN_4X.h5 (deflated 17%)


In [ ]:
from google.colab import files
files.download("/content/ESRCN_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## TEST

(1) Create necessary directories

In [5]:
!mkdir /content/results

In [8]:
!mkdir /content/espcn/logs

mkdir: cannot create directory ‘/content/espcn/logs’: File exists


(2) Define quality metric functions

In [ ]:
from skimage.metrics import structural_similarity as ssim

# define a function for peak signal-to-noise ratio (PSNR)
def psnr(target, ref):
         
    # assume RGB image
    target_data = target.astype(float)
    ref_data = ref.astype(float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(np.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

# define function for mean squared error (MSE)
def mse(target, ref):
    # the MSE between the two images is the sum of the squared difference between the two images
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    
    return err

# define function that combines all three image quality metrics
def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel =True))
    
    return scores

(3) Execute model on all images

In [ ]:
!python libs/espcn.py

(3) Print results \\
(make sure to adapt ref and degraded parts according to image read above)

In [ ]:
import math
import numpy as np
import cv2

ref = cv2.imread('/content/gdrive/MyDrive/MA/IVUSImages_PNG_Valid2/001.png')
degraded = cv2.imread('/content/gdrive/MyDrive/MA/IVUS_LowRes_Scale4_Valid/001.png')
output = cv2.imread('/content/results/001.png')
  
# image quality calculations
scores = []
h, w, _ = degraded.shape
degraded2 = cv2.resize(degraded, (4*w, 4*h))
scores.append(compare_images(degraded2, ref))

scores.append(compare_images(output, ref))

  
# print all scores for all images
print('Degraded Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[0][0], scores[0][1], scores[0][2]))
print('Reconstructed Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[1][0], scores[1][1], scores[1][2]))

Degraded Image: 
PSNR: 13.687345130800821
MSE: 8345.78327178955
SSIM: 0.505967748705135

Reconstructed Image: 
PSNR: 10.919700420902785
MSE: 15784.527042388916
SSIM: 0.11427748848404905

